# ⚠️ Before Running: Secret Scope Setup

Before running this notebook, you need to set up a secret scope and secrets using the Databricks CLI:

```bash
# Create the secret scope
databricks secrets create-scope --scope multai_scope

# Add secrets to the scope (do this for each secret)
databricks secrets put --scope multai_scope --key DATABRICKS_TOKEN
databricks secrets put --scope multai_scope --key YFINANCE_API_KEY  # if using premium API
```

Required Secrets in `multai_scope`:
- `DATABRICKS_TOKEN`: Your Databricks Personal Access Token
- `YFINANCE_API_KEY`: Yahoo Finance API key (if using premium API)

To verify the setup in this notebook, we'll check if we can access the secret scope (but not the secrets themselves for security).

In [ ]:
# Verify secret scope exists
def check_secret_scope():
    try:
        # List all secret scopes
        scopes = dbutils.secrets.listScopes()
        
        # Check if our scope exists
        scope_exists = any(scope.name == 'multai_scope' for scope in scopes)
        
        if scope_exists:
            print("✅ Secret scope 'multai_scope' is configured")
            # List keys in scope (this only shows key names, not values)
            keys = dbutils.secrets.list('multai_scope')
            print("\nConfigured secrets:")
            for key in keys:
                print(f"- {key.key}")
        else:
            print("❌ Secret scope 'multai_scope' not found!")
            print("Please run the CLI commands shown above to create the scope and add secrets.")
    except Exception as e:
        print(f"Error checking secret scope: {str(e)}")
        raise

# Run the check
check_secret_scope()

# Workspace Setup and Unity Catalog Configuration

This notebook:
1. Sets up Unity Catalog catalog and schemas
2. Verifies workspace configuration
3. Prints current user and workspace details

**Note**: Requires appropriate Unity Catalog permissions

## Check Current User and Workspace Details

In [ ]:
from pyspark.sql import SparkSession
import json

# Get current user
current_user = spark.sql("SELECT current_user()").collect()[0][0]
print(f"Current User: {current_user}")

# Get workspace details
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
print(f"Workspace URL: {workspace_url}")

# Print Spark version
print(f"Spark Version: {spark.version}")

## Create Unity Catalog and Schemas

Setting up the finance catalog with bronze and silver schemas for our medallion architecture.

In [ ]:
# Function to safely create catalog/schema
def create_if_not_exists(entity_type: str, name: str, comment: str = None):
    try:
        if entity_type.lower() == 'catalog':
            spark.sql(f"CREATE CATALOG IF NOT EXISTS {name}")
            if comment:
                spark.sql(f"ALTER CATALOG {name} SET COMMENT '{comment}'")
        elif entity_type.lower() == 'schema':
            spark.sql(f"CREATE SCHEMA IF NOT EXISTS {name}")
            if comment:
                spark.sql(f"ALTER SCHEMA {name} SET COMMENT '{comment}'")
        print(f"{entity_type.capitalize()} '{name}' created/verified successfully")
    except Exception as e:
        print(f"Error creating {entity_type} '{name}': {str(e)}")
        raise

In [ ]:
# Create finance catalog
create_if_not_exists(
    'catalog',
    'finance_catalog',
    'Catalog for portfolio management data and analytics'
)

# Set the finance catalog as default
spark.sql("USE CATALOG finance_catalog")

In [ ]:
# Create bronze schema
create_if_not_exists(
    'schema',
    'finance_catalog.bronze',
    'Raw financial data ingestion layer'
)

# Create silver schema
create_if_not_exists(
    'schema',
    'finance_catalog.silver',
    'Cleaned and transformed financial data'
)

## Verify Setup

List the created catalogs and schemas to verify the setup.

In [ ]:
# List available catalogs
print("Available Catalogs:")
display(spark.sql("SHOW CATALOGS"))

# List schemas in finance_catalog
print("\nSchemas in finance_catalog:")
display(spark.sql("SHOW SCHEMAS IN finance_catalog"))